# This Notbebook takes as input a graph, and outputs 

In [1]:
from VG_graphs import get_filtered_graphs, get_filtered_relationships, get_filtered_objects, get_filtered_attributes

filtered_graphs = get_filtered_graphs()
relationship_labels = get_filtered_relationships()
object_labels = get_filtered_objects()
attribute_labels = get_filtered_attributes()

Filtered graphs loaded from file
Filtered relationships loaded from file
Filtered objects loaded from file
Filtered attributes loaded from file


In [2]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
from VG_graphs.realistic_adversarial import convert_all_adversarially_realistic
from PIL import Image
import random
import torch

# set the random seed
random.seed(42)

shuffled_graphs = filtered_graphs.copy()
random.shuffle(shuffled_graphs)

image_dir = "/local/home/stuff/visual-genome/VG/"
metadata_path = "/local/home/jthomm/GraphCLIP/datasets/visual_genome/processed/"
try:
    curated_adversarial = torch.load(metadata_path + "curated_adversarial.pt") # a dict with image_id as key and a graph and the adversarial perturbations as value
    shuffled_graphs = [g for g in shuffled_graphs if g.image_id not in curated_adversarial.keys()]
except:
    curated_adversarial = {}

# Define your image list and corresponding options
image_list = [image_dir + f'{g.image_id}.jpg' for g in shuffled_graphs]
def options_list(index):
    g = shuffled_graphs[index]
    adv_perturbations = convert_all_adversarially_realistic(g, relationship_labels)
    res = []
    for adv_perturbation in adv_perturbations:
        graph_edge = adv_perturbation[0]
        subject = g.nodes[graph_edge[0]]['name']
        object = g.nodes[graph_edge[1]]['name']
        predicate = g.edges[graph_edge]['predicate']
        adv_predicates = adv_perturbation[1]
        for adv_predicate in adv_predicates:
            # res.append((f'{subject} {predicate} {object} -> {subject} {adv_predicate} {object}',(g,graph_edge,adv_predicate)))
            res.append((f'{subject} {adv_predicate} {object}',(g,graph_edge,adv_predicate)))
    return res[:15]


# Store the selections in an array
selections = []

def load_image(file_path):
    with open(file_path, "rb") as f:
        return f.read()

# Define the functions to handle button clicks
def on_next_button_click(button):
    # Get the selected options
    selected_options = [cb.metadata for cb in checkboxes if cb.value]
    
    # Store the selections along with the current image
    selections.append((current_image.value, selected_options))
    
    # Go to the next image
    current_index = image_list.index(current_image.value)
    while current_index + 1 < len(image_list):
        if shuffled_graphs[current_index].image_id in curated_adversarial:
            current_index += 1
            continue
        current_image.value = image_list[current_index + 1]
        current_image_widget.value = load_image(current_image.value)
        update_checkboxes()
        break
    if current_index + 1 == len(image_list):
        print("Finished!")
        print("Selections:", selections)
        next_button.disabled = True

def on_save_button_click(button):
    # first call on_next_button_click to save the last selection
    on_next_button_click(next_button)
    # save the selections to curated_adversarial.pt
    for selection in selections:
        image_id = int(selection[0].split('/')[-1].split('.')[0])
        if image_id not in curated_adversarial:
            curated_adversarial[image_id] = []
        curated_adversarial[image_id].extend(selection[1])
    torch.save(curated_adversarial, metadata_path + "curated_adversarial.pt")

def update_checkboxes():
    current_index = image_list.index(current_image.value)
    current_options = options_list(current_index)
    
    # Create new checkboxes
    new_checkboxes = [widgets.Checkbox(description=option[0], value=False) for option in current_options]
    for cb, option in zip(new_checkboxes, current_options):
        cb.metadata = option[1]
        
    # delete the old checkboxes from the VBox
    checkbox_vbox.children = tuple()
    # Update the VBox with the new checkboxes
    checkbox_vbox.children = tuple(new_checkboxes)
    global checkboxes
    # delete the old checkboxes
    for cb in checkboxes:
        cb.close()
    del checkboxes
    checkboxes = new_checkboxes

# Create widgets
current_image = widgets.Text(value=image_list[0], description='Image:', disabled=True)
current_image_widget = widgets.Image(value=load_image(image_list[0]), format='png', width=400, height=400)
checkboxes = [widgets.Checkbox(description=option[0], value=False) for option in options_list(0)]
for cb, option in zip(checkboxes, options_list(0)):
    cb.metadata = option[1]
checkbox_vbox = widgets.VBox()
next_button = widgets.Button(description="Next")
next_button.on_click(on_next_button_click)
save_button = widgets.Button(description="Save")
save_button.on_click(on_save_button_click)

# Display widgets
display(current_image)
display(current_image_widget)
display(checkbox_vbox)
update_checkboxes()
display(next_button)
display(save_button)


Text(value='/local/home/stuff/visual-genome/VG/402.jpg', description='Image:', disabled=True)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

VBox()

Button(description='Next', style=ButtonStyle())

Button(description='Save', style=ButtonStyle())

In [4]:
import torch
metadata_path = "/local/home/jthomm/GraphCLIP/datasets/visual_genome/processed/"
curated_adversarialt = torch.load(metadata_path + "curated_adversarial.pt") # a dict with image_id as key and a graph and the adversarial perturbations as value
curated_adversarialt = {k: v for k, v in curated_adversarialt.items() if len(v) > 0}
print(len(curated_adversarialt))


51


In [4]:
import VG_graphs.realistic_adversarial as ra 
dataset = ra.get_realistic_graphs_dataset()
print(dataset[0][0].image_id,dataset[0][1].image_id)

2335263 2335263
